In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
# import numpy as np
# import re, string
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import SnowballStemmer
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer

In [ ]:
df = pd.read_excel("../Scrapper_New/Data_Files/Final/CrimevsNonCrime.xlsx")
df_y = ((pd.Categorical(df.Label).codes).astype(int))
df.head

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df['Title'], df_y, test_size=0.2, random_state=42)
print(X_train)

In [ ]:
x=X_test.value_counts()
print(x)
sns.barplot(x=x.index,y=x)

 Dubey encounter: Ex-judge B S Chauhan to head inquiry panel, complete probe in 2 months, says SC                  2
James Gunn gets engaged to The Suicide Squad actor Jennifer Holland: ‘Happiness’                                   2
 32kg gold seized from Vande Bharat passengers in Jaipur                                                           2
 Day 32 of coronavirus lockdown: Ground report from Indian cities                                                  2
Explained: What we know about South Korea’s new president Yoon Suk-yeol, and his proposed policies                 2
                                                                                                                  ..
Rajasthan man kills wife, son with iron rod over personal dispute                                                  1
Jhund director Nagraj Manjule on Amitabh Bachchan’s influence on him: ‘Start of my love for cinema’                1
PSY’s much-awaited That That feat BTS’ Suga becomes a phenomenon

<AxesSubplot:ylabel='Title'>